In [1]:
import itertools
import random

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
date_csv = '20241122_130405' 
df_orig = pd.read_csv(f'../logs/rundirs/sorted-{date_csv}.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,20241122_113839,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,783.84,0,584.0,3,0,0,36,0,0,1,460.5,304.5,599.9,0.3,5.6,1.0
1,20241122_113951,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,783.84,3,2469.8,5,7,7,45,1,2,0,29.1,18.2,599.9,0.3,5.6,4.1
2,20241122_114110,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,783.84,3,2469.8,5,7,7,44,1,1,0,31.0,18.7,599.9,0.3,5.6,4.1
3,20241122_114229,map-generator/generated-maps/2024-11-22_11:27:...,0,HumanDrivenVehicle,779.77,1,888.0,8,0,0,41,0,0,0,341.6,136.8,599.9,0.3,5.6,1.5
4,20241122_114447,map-generator/generated-maps/2024-11-22_11:27:...,0,HumanDrivenVehicle,779.77,3,2469.3,6,8,8,44,0,3,0,26.5,12.5,599.9,0.3,5.6,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,20241122_125429,map-generator/generated-maps/2024-11-22_11:26:...,4,AutonomousVehicle,604.25,2,2815.7,5,0,0,13,0,1,0,14.8,6.3,599.9,0.3,5.6,4.7
296,20241122_125600,map-generator/generated-maps/2024-11-22_11:26:...,4,AutonomousVehicle,604.25,1,1741.7,4,0,0,15,0,0,0,221.0,126.5,599.9,0.3,5.6,2.9
297,20241122_125729,map-generator/generated-maps/2024-11-22_11:27:...,4,AutonomousVehicle,604.12,2,2879.2,3,0,0,14,0,0,0,31.9,30.4,599.9,0.3,5.6,4.8
298,20241122_125905,map-generator/generated-maps/2024-11-22_11:27:...,4,AutonomousVehicle,604.12,2,2719.6,5,0,0,14,0,1,0,37.9,20.8,599.9,0.3,5.6,4.5


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id,
    df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/[^/]+$', expand=True),
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+).json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,dir_map,i_map,Positions variant,probabilityForcingForHuman,are_bridges
0,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,1,1,0.0,True
1,map-generator/generated-maps/2024-11-22_11:26:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,1,1,1.0,True
2,map-generator/generated-maps/2024-11-22_11:26:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,1,1,1.0,True
3,map-generator/generated-maps/2024-11-22_11:27:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,1,1,0.0,False
4,map-generator/generated-maps/2024-11-22_11:27:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,1,1,1.0,False
...,...,...,...,...,...,...,...,...,...,...
295,map-generator/generated-maps/2024-11-22_11:26:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,1,10,1.0,True
296,map-generator/generated-maps/2024-11-22_11:26:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,1,10,1.0,True
297,map-generator/generated-maps/2024-11-22_11:27:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,1,10,0.0,False
298,map-generator/generated-maps/2024-11-22_11:27:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,1,10,1.0,False


In [4]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([df_id[['are_bridges', 'Coordination strategy', 'probabilityForcingForHuman', 'Positions variant']], df_orig.iloc[:, index_id + 1:]], axis=1)
df

,are_bridges,Coordination strategy,probabilityForcingForHuman,Positions variant,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,...,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,True,baseline,0.0,1,0,HumanDrivenVehicle,783.84,0,584.0,3,...,36,0,0,1,460.5,304.5,599.9,0.3,5.6,1.0
1,True,change of priorities,1.0,1,0,HumanDrivenVehicle,783.84,3,2469.8,5,...,45,1,2,0,29.1,18.2,599.9,0.3,5.6,4.1
2,True,stops,1.0,1,0,HumanDrivenVehicle,783.84,3,2469.8,5,...,44,1,1,0,31.0,18.7,599.9,0.3,5.6,4.1
3,False,baseline,0.0,1,0,HumanDrivenVehicle,779.77,1,888.0,8,...,41,0,0,0,341.6,136.8,599.9,0.3,5.6,1.5
4,False,change of priorities,1.0,1,0,HumanDrivenVehicle,779.77,3,2469.3,6,...,44,0,3,0,26.5,12.5,599.9,0.3,5.6,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,True,change of priorities,1.0,10,4,AutonomousVehicle,604.25,2,2815.7,5,...,13,0,1,0,14.8,6.3,599.9,0.3,5.6,4.7
296,True,stops,1.0,10,4,AutonomousVehicle,604.25,1,1741.7,4,...,15,0,0,0,221.0,126.5,599.9,0.3,5.6,2.9
297,False,baseline,0.0,10,4,AutonomousVehicle,604.12,2,2879.2,3,...,14,0,0,0,31.9,30.4,599.9,0.3,5.6,4.8
298,False,change of priorities,1.0,10,4,AutonomousVehicle,604.12,2,2719.6,5,...,14,0,1,0,37.9,20.8,599.9,0.3,5.6,4.5


In [5]:
series_blocks = df[df['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['Positions variant'])['Is blocked'].sum()
indices_locations_blocked = series_blocks[series_blocks != 0].index
indices_locations_nonblocked = series_blocks[series_blocks == 0].index
indices_locations_nonblocked

Index([2, 3, 6, 8, 10], dtype='int64', name='Positions variant')

In [6]:
def save_and_show(basename):  # to avoid inlining large image data into the notebook file
    filename = f'images/{basename}.png'
    plt.savefig(filename)
    plt.close()
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    

def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, is_aut):    
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    
    column_y1 = 'Total distance traveled (m)'
    color_y1 = 'tab:blue'
    
    columns_y2_aut = ['No. of collisions', 'No. of near-misses']
    colors_y2_aut = ['red', 'yellow']
    
    columns_y2_hum = ['No. of violations'] + columns_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    if is_aut:
        columns_y2 = columns_y2_aut
        colors_y2 = colors_y2_aut
        title1 = 'Automated vehicles (summarised for AV1-AV4)' 
    else:
        columns_y2 = columns_y2_hum
        colors_y2 = colors_y2_hum
        title1 = 'Human-driven vehicle'
    
    # Get the global min and max values for consistent y-axis scaling
    y1_max = max(dfx[column_y1].max() for dfx in dfs_y1)
    y2_max = max(dfx[column].max()
                 for dfx in dfs_y2
                 for column in set(columns_y2_aut + columns_y2_hum))
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars
        handles = [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel(column_y1, color=color_y1)
        ax.tick_params(axis='y', labelcolor=color_y1)
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points
        handles += [ax_right.plot(x_positions, df_pos[column], label=column,
                                  marker='o', linestyle='', color=color)[0]
                    for column, color in zip(columns_y2, colors_y2)]
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
    fig.suptitle(f'{title1}\n{title2}', fontsize=16)
    fig.legend(handles=handles, labels=[column_y1, *columns_y2], ncol=len(handles), loc='upper right')
    plt.tight_layout()
    save_and_show(f'{title1}: {title2}')
    #print(id(df))
    #IPython.display.display(df)


key2df = {}
for is_aut in True, False:
    for are_bridges in True, False:
        dfx = df[df['are_bridges'] == are_bridges]
        dfx = dfx[dfx['Positions variant'].isin(indices_locations_nonblocked)]
        dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
        dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).sum()
        key2df[is_aut, are_bridges] = dfx    
        
for is_aut in True, False:
    for are_bridges in True, False:
        try:
            plot_aut_hum(key2df[is_aut, are_bridges], 
                         title2='With bridges' if are_bridges else 'Without bridges', 
                         dfs_y1=[dfx for (is_aut_dfx, _), dfx in key2df.items() if is_aut_dfx == is_aut], 
                         dfs_y2=key2df.values(), 
                         is_aut=is_aut)
        except Exception as exc:
            print(exc)
            #raise exc
    print()

In [7]:
def show_maps(title, indices, ncols): 
    nrows = max(1, (len(indices) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(indices)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, i_pos in zip(axes, indices):
        image = plt.imread(f'../screenshotting/screenshots/cropped/{i_pos}.png')
        ax.imshow(image)
        ax.title.set_text(f'Positions variant {i_pos}')
    
    fig.suptitle(title, fontsize=16)

    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    plt.tight_layout()
    save_and_show(title)
    
    
show_maps('Non-blocked', indices_locations_nonblocked, 4)
show_maps('Blocked', indices_locations_blocked, 4)